### importing the necessary libraries

In [7]:
import pyspark
import pyarrow

In [8]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import when, lit, col
from pyspark.sql import functions as func
from pyspark.sql.functions import col, isnan, when, count
from functools import reduce
from pyspark.sql.types import StructType, StructField, StringType, LongType, DateType, IntegerType

### creating sparksession object

In [9]:
spark =SparkSession.builder.appName('DecemberCps').getOrCreate()

23/03/21 16:09:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### reading the dataset into the environment 

In [10]:
df_cps = spark.read.csv("dec17pub.dat")

In [11]:
df_cps.show(10)

+--------------------+
|                 _c0|
+--------------------+
|00000479511071912...|
|00000479511071912...|
|00007169100494112...|
|00007169100494112...|
|00007169100494112...|
|00011017798798612...|
|00011017798798612...|
|00011020659338112...|
|00011028481568012...|
|00011032785646912...|
+--------------------+
only showing top 10 rows



### extracting the required columns from the dataset using the data dictionary file

In [12]:
df_cps1 = df_cps.select(func.concat_ws("",func.substring("_c0",1,15),func.substring("_c0", 71,5)).alias("householdId"),\
                                      func.concat_ws("/",func.substring("_c0",18,4),func.date_format(func.to_date(func.substring("_c0",16,2), "MM"),"MMM")).alias("interviewTime"),\
                                      func.substring("_c0", 24,3).alias("finalOutcome"),\
                                      func.substring("_c0", 31,2).alias("housingType"),\
                                      func.substring("_c0", 61,2).alias("householdType"),\
                                      func.substring("_c0", 33,2).alias("telephoneInHousehold"),\
                                      func.substring("_c0", 35,2).alias("accessToTelephoneElsewhere"),\
                                      func.substring("_c0", 37,2).alias("acceptableTelephoneInterview"),\
                                      func.substring("_c0", 65,2).alias("interviewType"),\
                                      func.substring("_c0", 39,2).alias("familyIncomeRange"),\
                                      func.substring("_c0", 91,1).alias("geoDivision"),\
                                      func.substring("_c0", 139,2).alias("race"))            

### showing ten(10) samples 

In [13]:
df_cps1.show(10)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|                        -1|                           1|            2|                9|          6|   1|
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|                        -1|                           1|            2|                9|        

### checking for number of rows in dataset

In [14]:
df_cps1.count()


146456

### printing schema of dataframe

In [15]:
df_cps1.printSchema()

root
 |-- householdId: string (nullable = false)
 |-- interviewTime: string (nullable = false)
 |-- finalOutcome: string (nullable = true)
 |-- housingType: string (nullable = true)
 |-- householdType: string (nullable = true)
 |-- telephoneInHousehold: string (nullable = true)
 |-- accessToTelephoneElsewhere: string (nullable = true)
 |-- acceptableTelephoneInterview: string (nullable = true)
 |-- interviewType: string (nullable = true)
 |-- familyIncomeRange: string (nullable = true)
 |-- geoDivision: string (nullable = true)
 |-- race: string (nullable = true)



### checking for null values 

In [16]:
sum(df_cps1.where(reduce(lambda x, y: x | y, (func.col(x).isNull() for x in df_cps1.columns))))

Column<'((((((((((((householdId + 0) + interviewTime) + finalOutcome) + housingType) + householdType) + telephoneInHousehold) + accessToTelephoneElsewhere) + acceptableTelephoneInterview) + interviewType) + familyIncomeRange) + geoDivision) + race)'>

### Checking for missing values

In [17]:
for x in df_cps1.columns:
    list_of_values = df_cps1.select([count(when(isnan(x) | col(x).isNull(), x))]).collect()
print(list_of_values)

[Row(count(CASE WHEN (isnan(race) OR (race IS NULL)) THEN race END)=0)]


## Decoding the required encoded columns

### decoding 'family income range'

In [18]:
new_df = df_cps1.withColumn("new_FamilyIncomeRange",
                                when(col("familyIncomeRange")=="1", lit("0-5000")).
                                when(col("familyIncomeRange")=="2", lit("5000-7499")).
                                when(col("familyIncomeRange")=="3", lit("7500-9999")).
                                when(col("familyIncomeRange")=="4", lit("10000-12499")).
                                when(col("familyIncomeRange")=="5", lit("12500-14499")).
                                when(col("familyIncomeRange")=="6", lit("15000-19999")).
                                when(col("familyIncomeRange")=="7", lit("20000-24999")).
                                when(col("familyIncomeRange")=="8", lit("25000-29999")).
                                when(col("familyIncomeRange")=="9", lit("30000-34999")).
                                when(col("familyIncomeRange")=="10",lit("35000-39999")).
                                when(col("familyIncomeRange")=="11",lit("40000-49999")).
                                when(col("familyIncomeRange")=="12",lit("50000-59999")).
                                when(col("familyIncomeRange")=="13",lit("60000-74999")).
                                when(col("familyIncomeRange")=="14",lit("75000-99999")).
                                when(col("familyIncomeRange")=="15",lit("100000-149999")).
                                when(col("familyIncomeRange")=="16",lit("150000...")).
                                otherwise(lit("not_applicable")))


In [19]:
new_df.show(5)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|new_FamilyIncomeRange|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|                        -1|                           1|            2|                9|          6|   1|       not_applicable|
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|        

### decoding 'geographical division'

In [20]:
new_df = new_df.withColumn("new_geoDivision", 
                            when(col("geoDivision") == "1", lit("New England")).
                            when(col("geoDivision") == "2", lit("Middle Atlantic")).
                            when(col("geoDivision") == "3", lit("East North Central")).
                            when(col("geoDivision") == "4", lit("West North Central")).
                            when(col("geoDivision") == "5", lit("South Atlantic")).
                            when(col("geoDivision") == "6", lit("East South Central")).
                            when(col("geoDivision") == "7", lit("West South Central")).
                            when(col("geoDivision") == "8", lit("Mountain")).
                            when(col("geoDivision") == "9", lit("Pacific")).
                            otherwise(lit("not_applicable")))

In [21]:
new_df.show(5)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|new_FamilyIncomeRange|   new_geoDivision|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|                        -1|                           1|            2|                9|          6|   1|       not_applicable|East South Central|
|00000479511071906011|     2

### decoding 'race'

In [22]:
new_df=new_df.withColumn("new_Race",
                           when(col("race") == " 1", lit("White Only")).
                           when(col("race") == " 2", lit("Black Only")).
                           when(col("race") == " 3", lit("American Indian,Alaskan Native Only")).
                           when(col("race") == " 4", lit("Asian Only")).
                           when(col("race") == " 5", lit("Hawaiian/Pacific Islander Only")).
                           when(col("race") == " 6", lit("White-Black")).
                           when(col("race") == " 7", lit("White-AI")).
                           when(col("race") == " 8", lit("White-Asian")).
                           when(col("race") == " 9", lit("White-HP")).
                           when(col("race") == "10", lit("Black-AI")).
                           when(col("race") == "11", lit("Black-Asian")).
                           when(col("race") == "12", lit("Black-HP")).
                           when(col("race") == "13", lit("AI-Asian")).
                           when(col("race") == "14", lit("AI-HP")).
                           when(col("race") == "15", lit("Asian-HP")).
                           when(col("race") == "16", lit("W-B-AI")).
                           when(col("race") == "17", lit("W-B-A")).
                           when(col("race") == "18", lit("W-B-HP")).
                           when(col("race") == "19", lit("W-AI-A")).
                           when(col("race") == "20", lit("W-AI-HP")).
                           when(col("race") == "21", lit("W-A-HP")).
                           when(col("race") == "22", lit("B-AI-A")).
                           when(col("race") == "23", lit("W-B-AI-A")).
                           when(col("race") == "24", lit("W-AI-A-HP")).
                           when(col("race") == "25", lit("Other 3 Race Combinations")).
                           when(col("race") == "26", lit("Other 4 and 5 Race Combinations")).
                           otherwise(lit("not_applicable")))

In [23]:
new_df.show(5)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|new_FamilyIncomeRange|   new_geoDivision|  new_Race|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|                        -1|                           1|            2|                9|          6|   1|       not_applicable|East South Cent

### decoding 'household has a telephone'

In [24]:
new_df = new_df.withColumn("new_telephoneInHousehold",
                          when(col("telephoneInHousehold") == " 1", lit("Yes")).
                          when(col("telephoneInHousehold") == " 2", lit("No")).
                          otherwise(lit("not_applicable")))
                           

In [25]:
new_df.show(5)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+------------------------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|new_FamilyIncomeRange|   new_geoDivision|  new_Race|new_telephoneInHousehold|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+------------------------+
|00000479511071906011|     2017/Dec|         201|          1|            1|                   1|                        -1|                           1|           

### decoding 'household can access a telephone elsewhere'

In [26]:
new_df = new_df.withColumn("new_AccessToTelephoneElsewhere",
                          when(col("accessToTelephoneElsewhere") == " 1", lit("Yes")).
                          when(col("accessToTelephoneElsewhere") == " 2", lit("No")).
                          otherwise(lit("not_applicable")))

In [27]:
new_df.show(5)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+------------------------+------------------------------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|new_FamilyIncomeRange|   new_geoDivision|  new_Race|new_telephoneInHousehold|new_AccessToTelephoneElsewhere|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+------------------------+------------------------------+
|00000479511071906011|     2017/Dec|         201|          1|         

### decoding 'Is telephone interview acceptable'

In [28]:
new_df = new_df.withColumn("new_acceptableTelephoneInterview",
                          when(col("acceptableTelephoneInterview") == " 1", lit("Yes")).
                          otherwise(lit("not_applicable")))

In [29]:
new_df.show(5)

+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+------------------------+------------------------------+--------------------------------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|telephoneInHousehold|accessToTelephoneElsewhere|acceptableTelephoneInterview|interviewType|familyIncomeRange|geoDivision|race|new_FamilyIncomeRange|   new_geoDivision|  new_Race|new_telephoneInHousehold|new_AccessToTelephoneElsewhere|new_acceptableTelephoneInterview|
+--------------------+-------------+------------+-----------+-------------+--------------------+--------------------------+----------------------------+-------------+-----------------+-----------+----+---------------------+------------------+----------+------------------------+------------------------------+-----

### dropping the encoded columns

In [30]:
new_ddf = new_df.drop("familyIncomeRange","geoDivision","race","telephoneInHousehold","accessToTelephoneElsewhere","acceptableTelephoneInterview")

In [31]:
new_ddf.show(5)

+--------------------+-------------+------------+-----------+-------------+-------------+---------------------+------------------+----------+------------------------+------------------------------+--------------------------------+
|         householdId|interviewTime|finalOutcome|housingType|householdType|interviewType|new_FamilyIncomeRange|   new_geoDivision|  new_Race|new_telephoneInHousehold|new_AccessToTelephoneElsewhere|new_acceptableTelephoneInterview|
+--------------------+-------------+------------+-----------+-------------+-------------+---------------------+------------------+----------+------------------------+------------------------------+--------------------------------+
|00000479511071906011|     2017/Dec|         201|          1|            1|            2|       not_applicable|East South Central|White Only|                     Yes|                not_applicable|                             Yes|
|00000479511071906011|     2017/Dec|         201|          1|            1| 

## Question1


### Selecting columns that are important for the analysis

In [32]:
responders_per_family_income_range_df = new_ddf.select("householdId","new_FamilyIncomeRange")
responders_per_family_income_range_df.show(5)

+--------------------+---------------------+
|         householdId|new_FamilyIncomeRange|
+--------------------+---------------------+
|00000479511071906011|       not_applicable|
|00000479511071906011|       not_applicable|
|00007169100494106111|          40000-49999|
|00007169100494106111|          40000-49999|
|00007169100494106111|          40000-49999|
+--------------------+---------------------+
only showing top 5 rows



### Grouping and sorting in other to get the top 10 count based on householdId

In [38]:
df_cps = responders_per_family_income_range_df.groupBy("householdId","new_FamilyIncomeRange").count()
top_ten_fam = df_cps.sort(col("count").desc())

### Converting into pandas in order to visualize using pandas method

In [39]:
pandasDF = top_ten_fam.to_pandas_on_spark()
pandasDF.head(10)

/mnt/spark/python/pyspark/sql/dataframe.py:3313: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


,householdId,new_FamilyIncomeRange,count
0,17136002800228507011,150000...,14
1,65495620125100906011,40000-49999,13
2,20694090797100407011,75000-99999,13
3,16002611040711006011,150000...,12
4,41020360025347607011,60000-74999,12
5,20061136481416406011,50000-59999,12
6,01051409682959106011,150000...,12
7,78548093966100306011,75000-99999,12
8,03091015830821906011,35000-39999,12
9,05031242797854106011,not_applicable,12


### Question 2

### Getting the count of responders per geographical division as regards race

In [40]:
responders_per_race = new_ddf.select("new_geoDivision","new_race")
responders_per_race.show()

+------------------+--------------+
|   new_geoDivision|      new_race|
+------------------+--------------+
|East South Central|    White Only|
|East South Central|    White Only|
|East South Central|    White Only|
|East South Central|    White Only|
|East South Central|    White Only|
|East South Central|    Black Only|
|East South Central|    Black Only|
|East South Central|not_applicable|
|East South Central|    White Only|
|East South Central|    Black Only|
|East South Central|    Black Only|
|East South Central|    Black Only|
|East South Central|    Black Only|
|East South Central|    Black Only|
|East South Central|    White Only|
|East South Central|    White Only|
|East South Central|    Black Only|
|East South Central|    Black Only|
|East South Central|    White Only|
|East South Central|    White Only|
+------------------+--------------+
only showing top 20 rows



### Grouping and sorting in other to get the top 10 count based on geographical location

In [41]:
responders_per_geo_loc_div_race = responders_per_race.groupBy("new_geoDivision", "new_race").count()
top_ten_geo = responders_per_geo_loc_div_race.sort(col("count").desc())
top_ten_geo.show(10)

+------------------+----------+-----+
|   new_geoDivision|  new_race|count|
+------------------+----------+-----+
|    South Atlantic|White Only|16999|
|          Mountain|White Only|14343|
|           Pacific|White Only|13214|
|East North Central|White Only|11325|
|West South Central|White Only|11248|
|West North Central|White Only| 9884|
|   Middle Atlantic|White Only| 8487|
|       New England|White Only| 8410|
|East South Central|White Only| 6580|
|    South Atlantic|Black Only| 4899|
+------------------+----------+-----+
only showing top 10 rows



## Question 3
### How many responders do not have telephone in their house, but can access a telephone elsewhere and telephone interview is accepted (show top 10)?

In [43]:
question3 = new_ddf.select("householdId", "new_telephoneInHousehold", "new_AccessToTelephoneElsewhere", "new_acceptableTelephoneInterview")
question3.show(10)

+--------------------+------------------------+------------------------------+--------------------------------+
|         householdId|new_telephoneInHousehold|new_AccessToTelephoneElsewhere|new_acceptableTelephoneInterview|
+--------------------+------------------------+------------------------------+--------------------------------+
|00000479511071906011|                     Yes|                not_applicable|                             Yes|
|00000479511071906011|                     Yes|                not_applicable|                             Yes|
|00007169100494106111|                     Yes|                not_applicable|                             Yes|
|00007169100494106111|                     Yes|                not_applicable|                             Yes|
|00007169100494106111|                     Yes|                not_applicable|                             Yes|
|00011017798798608011|                     Yes|                not_applicable|                          

### Filtering, grouping and taken the counts of rows in descending order 

In [48]:
resp_no_access_phone = question3.filter((question3.new_telephoneInHousehold == "No") &
                                        (question3.new_AccessToTelephoneElsewhere == "Yes") &
                                        (question3.new_acceptableTelephoneInterview == "Yes"))
resp_no_access_phone.show()

resp_no_access_phone1 = resp_no_access_phone.groupBy("householdId",
                                         "new_acceptableTelephoneInterview", 
                                         "new_AccessToTelephoneElsewhere", 
                                         "new_telephoneInHousehold").count()

top_ten_fam = resp_no_access_phone1.sort(col("count").desc())

+--------------------+------------------------+------------------------------+--------------------------------+
|         householdId|new_telephoneInHousehold|new_AccessToTelephoneElsewhere|new_acceptableTelephoneInterview|
+--------------------+------------------------+------------------------------+--------------------------------+
|18082060959110006012|                      No|                           Yes|                             Yes|
|19805189600020106011|                      No|                           Yes|                             Yes|
|03161060105100506011|                      No|                           Yes|                             Yes|
|10016053056110006011|                      No|                           Yes|                             Yes|
|28171000420025707111|                      No|                           Yes|                             Yes|
|28171000420025707111|                      No|                           Yes|                          

### Converting to pandas dataframe

In [49]:
pdData = top_ten_fam.toPandas()

filter_item = pdData[(pdData.new_telephoneInHousehold == "No") & (pdData.new_AccessToTelephoneElsewhere == "Yes") & (pdData.new_acceptableTelephoneInterview == "Yes")]
filter_item.head(10)

,householdId,new_acceptableTelephoneInterview,new_AccessToTelephoneElsewhere,new_telephoneInHousehold,count
0,13503264410435106011,Yes,Yes,No,8
1,08119602254072607011,Yes,Yes,No,7
2,28171000420025707111,Yes,Yes,No,6
3,90630208491072106011,Yes,Yes,No,6
4,97817059269120406011,Yes,Yes,No,6
5,41000278125151206011,Yes,Yes,No,5
6,81000414076269607011,Yes,Yes,No,5
7,96430076554110007011,Yes,Yes,No,5
8,10196319080002406011,Yes,Yes,No,5
9,13169102010096006011,Yes,Yes,No,5


## Question 4
### How many responders can access to a telephone, but telephone interview is not accepted (show top 10)?

In [50]:
question4 = new_ddf.select("householdId", "new_telephoneInHousehold", "new_AccessToTelephoneElsewhere", "new_acceptableTelephoneInterview")
question4.show(10)

+--------------------+------------------------+------------------------------+--------------------------------+
|         householdId|new_telephoneInHousehold|new_AccessToTelephoneElsewhere|new_acceptableTelephoneInterview|
+--------------------+------------------------+------------------------------+--------------------------------+
|00000479511071906011|                     Yes|                not_applicable|                             Yes|
|00000479511071906011|                     Yes|                not_applicable|                             Yes|
|00007169100494106111|                     Yes|                not_applicable|                             Yes|
|00007169100494106111|                     Yes|                not_applicable|                             Yes|
|00007169100494106111|                     Yes|                not_applicable|                             Yes|
|00011017798798608011|                     Yes|                not_applicable|                          

In [58]:
resp_no_access_phone2 = question4.filter((question4.new_telephoneInHousehold == "Yes") |
                                        (question4.new_AccessToTelephoneElsewhere == "Yes") &
                                        (question4.new_acceptableTelephoneInterview == "No"))


resp_no_access_phone2 = resp_no_access_phone2.groupBy("householdId",
                                         "new_telephoneInHousehold",
                                         "new_acceptableTelephoneInterview",             
                                         "new_AccessToTelephoneElsewhere").count()

top_ten_fam = resp_no_access_phone2.sort(col("count").desc()).show(10)

+--------------------+------------------------+--------------------------------+------------------------------+-----+
|         householdId|new_telephoneInHousehold|new_acceptableTelephoneInterview|new_AccessToTelephoneElsewhere|count|
+--------------------+------------------------+--------------------------------+------------------------------+-----+
|17136002800228507011|                     Yes|                             Yes|                not_applicable|   14|
|65495620125100906011|                     Yes|                  not_applicable|                           Yes|   13|
|01051409682959106011|                     Yes|                  not_applicable|                not_applicable|   12|
|16002611040711006011|                     Yes|                             Yes|                not_applicable|   12|
|78548093966100306011|                     Yes|                  not_applicable|                not_applicable|   12|
|05031242797854106011|                     Yes|         

***                                             THE END                                                         ***